#Imports and Google drive access

In [1]:
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
#from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer,WordNetLemmatizer,SnowballStemmer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,LSTM,MaxPool1D,Dense,Embedding,Input,InputLayer,Flatten,UpSampling1D,Lambda,Reshape,BatchNormalization,ZeroPadding1D,GRU
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.models import Model
from keras.engine.sequential import Sequential
import numpy as np
import json
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from wordcloud import WordCloud
from sklearn import svm
from sklearn.utils import shuffle
import mimetypes as mim
from ast import literal_eval
from sklearn.linear_model import LogisticRegressionCV,SGDClassifier,RidgeClassifierCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.neural_network import MLPClassifier,multilayer_perceptron
from sklearn.model_selection import StratifiedKFold,StratifiedShuffleSplit,GridSearchCV
import zipfile as zpf
import xgboost as xgb
from xgboost import XGBClassifier
import pandas as pd
import gensim
from PIL import Image , ImageFile
from sklearn.naive_bayes import MultinomialNB
# Adding new types t
mim.add_type(ext='.go',type='text')
mim.add_type(ext='.js',type='text')
mim.add_type(ext='.json',type='text')
import nltk
nltk.download('wordnet')
!pip3 install pytesseract
import pytesseract

Using TensorFlow backend.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
  Created wheel for pytesseract: filename=pytesseract-0.3.0-py2.py3-none-any.whl size=20970 sha256=46d858f007295fd9f40c5d64059b3c43e3782b8ee4edd89f469af252561c8f01
  Stored in directory: /root/.cache/pip/wheels/78/c9/ac/4cb76bd547f9970070522439e5203ba7926c5c5c4f131583ea
Successfully built pytesseract


In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/'My Drive'/'Colab Notebooks'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/Colab Notebooks


# Data Preparation

In [0]:
# Loading Data 
root ='/gdrive/My Drive/Colab Notebooks/newData'
repoNamesList=[]
paths=[]
pathNamesList=[]
for pathName in os.walk(root):
        paths+=[pathName]


In [0]:
#pathsWithFile=paths[2::]
#type(pathsWithFile)
#paths[0][1].remove(paths[0][1][3])

In [4]:
labels=paths[0][1]
#labels=np.array(labels)
#labels.reshape(-1,1)
labels

['transportation',
 'Technology',
 'Retail',
 'Real_estate',
 'media',
 'Law',
 'Insurance',
 'Healthcare',
 'Food_And_beverages',
 'FinTech',
 'Energy',
 'Education',
 'Banking',
 'Automotive']

In [0]:
Y=[]
reposNameList=[]
reposContentsList=[]
contentMerged = ''

In [0]:
root = '/gdrive/My Drive/Colab Notebooks/newData'
cntRepo = 0
for label in labels: 
    cntRepo = 0
    for repoName in os.listdir(path='./newData/'+label):
        if(cntRepo <= 140):
              Y+=[label]
              filesContent=open('./newData/'+label+'/'+repoName,'r')
              contentDic=literal_eval(filesContent.read())
              contentMerged = ''
              for filesKey in contentDic[repoName]:
                    contentMerged+=repoName
                    contentMerged+=' ' 
                    contentMerged+= contentDic[repoName][filesKey]
              reposContentsList+=[contentMerged]  
              reposNameList+=[repoName]      
              filesContent.close() 
        cntRepo+=1
        #path=get_all_files('./newData/'+label+'/'+l)

In [7]:
print(len(reposNameList))
contentMerged=''
print(len(Y))
len(reposContentsList)

1917
1917


1917

In [0]:
file_1 = open('/gdrive/My Drive/Colab Notebooks/stemmed_lemmarized_data.txt','r')
cleanedData=literal_eval(file_1.read())

In [9]:
#print(reposNameList[0])
print(len(cleanedData))
#reposContentsList[0]

1917


# Data Preparation for data will be downloaded from Github and will be used to take each repo text and will be put in one file.



In [0]:
'''
# use the paths string to open files and collect the string of each repo under the specific topic like Automotive,etc and save the string into one file.
# Clean the data initialy because we will try afterthat to remove some unimportant files.
cleanedData=''
checkAlpha=0 
root ='newData'
for label in  pathsDic:  
    repoDic2={}
    if(os.path.isdir(os.path.join(root,label))==False):
        os.mkdir(os.path.join(root,label))    
    for repoName in  pathsDic[label]:
        repoDic2[repoName]={}  
        #print(pathsDic[label][repoName])
        for path in  pathsDic[label][repoName].split():
                #print(path)   
                typeGuess = mim.guess_type(path)
                if(typeGuess[0] is not None):
                    typeGuess=typeGuess[0].split('/')
                    checkImg=check_img(typeGuess)
                    checkText=check_text(typeGuess)
                    if(checkImg):  
                        #print(fileBytes)
                        fileName=path.split('/')[-1]
                        if(fileName.split('.')[-1] == 'SVG' or  fileName.split('.')[-1] == 'svg'):
                             continue
                        #Extract the text from the image.
                        if(fileName.split('.')[-1]=='png'or fileName.split('.')[-1]=='PNG'):
                            try:
                                imgFile=Image.open(path,'r').convert('RGB')
                            except:
                                continue
                            imgFile.save('ImgToSave.jpg')
                            fileName='ImgToSave.jpg'
                            try:
                                textOfImg=pytesseract.image_to_string(imread(fileName))
                                textOfImg=data_clean(textOfImg)
                            except:
                                   continue    
                        else:
                            try:
                                textOfImg=pytesseract.image_to_string(imread(path))
                                textOfImg=data_clean(textOfImg)
                            except:
                                   continue 
                        imgFile.close()        
                        repoDic2[repoName][path.split('/')[-1]]=textOfImg
                       
                    elif(checkText):
                        try:
                            file=open(path,'r')
                            lines=file.readlines()
                        except:
                            print(path)
                            continue   
                        cleanedData=data_clean(lines)         
                        repoDic2[repoName][path.split('/')[-1]]=cleanedData
                        
                elif(re.search('description',path)or re.search('README',path) or re.search('readme',path)):
                    try:
                        file=open(path,'r')      
                        lines=file.readlines()
                    except:
                        print(path)
                        continue
                    cleanedData=data_clean(lines)
                    repoDic2[repoName][path.split('/')[-1]]=cleanedData
        repoFile = open(os.path.join(root,os.path.join(label,repoName)),'w')
        repoFile.write(str(repoDic2))
        repoFile.close()
        repoDic2={}
 '''
'''
def data_clean(lines):
    cummulateString=''
    checkAlpha = 0
    for line in lines:
        line=line.replace('\\n',' ') 
        line=line.replace('.',' ')
        line=line.replace(':',' ')
        line=line.replace(';',' ')
        line=line.replace('(',' ')
        for char in line: 
            #Exclude the non alphabitical data.
            #print(char)
            if(char.isalpha() or char.isupper()):
                char = char.lower()
                cummulateString+=char
                checkAlpha=1
                #print(char)
            elif(char.isspace()):
                if(checkAlpha == 1):
                        cummulateString+=' '
                        checkAlpha=0 
                else:
                        cummulateString+=''
               
            else:
                cummulateString+=''
    cummulateString+=' ' 
    return cummulateString
def check_text(typeGuess):
    if(re.search(typeGuess[0],'text')):
         return True
    else:
         return False  
 def check_img(typeGuess):
    if(re.search(typeGuess[0],'image')):
         return True
    else:
         return False         

'''   
'''
repoDic2={}    
root ='newData'
for label in repoDic:   
    if(os.path.isdir(os.path.join(root,label))==False):
        os.mkdir(os.path.join(root,label))
    for repoName in repoDic[label]:
        repoDic2[repoName]={}  
        if(os.path.isdir(label+"/"+repoName)):
              os.mkdir(os.path.join(label,repoName))
        for path in repoDic[label][repoName]:
            path=path.split()
            print(path)
'''     

#Data Cleaning




In [0]:
# Remove space more than one.
for indx in range(0,len(reposContentsList)):
     reposContentsList[indx]=' '.join(reposContentsList[indx].split())

In [0]:
# Clean data from the extensions.
ext=['ymal','png','jpg','jpeg','js','src','java','xml','com','jar','html','yml','main','test','css','go','gif','if','let','https','http','foreach','for','while','break','continue','var','echo','int','!','"','#','$','%','&','(',',',')','+','-','.','/',':',';','<','=','>','?','@','[','\\'']','^','_','`','{','|','}''~','\t','\n']
for indx in range(0,len(reposContentsList)):
     for indx2 in range(0,len(ext)):
          reposContentsList[indx]=reposContentsList[indx].replace(ext[indx2],"")
     

In [0]:
# We add each file name of repo as the content to the string file of repo. So we try to clean each filename to help us to tokenize the file name and use it in training period.
signs=['!','"','#','$','%','&','(',',',')','+','-','.','/',':',';','<','=','>','?','@','[','\\'']','^','_','`','{','|','}''~','\t','\n']
for indx in range(0,len(reposContentsList)):
     for indx2 in range(0,len(signs)):
          reposContentsList[indx]=reposContentsList[indx].replace(signs[indx2]," ")

In [0]:
# For more Cleaning.Filter that data from the reserved words in Programming Language like c,c++,python.
progLangResvWordsDf =pd.read_csv('/gdrive/My Drive/Colab Notebooks/pl_key.csv')
progLangResvWordsLs=list(set(progLangResvWordsDf['key']))
for indx,word in enumerate(progLangResvWordsLs):
    progLangResvWordsLs[indx] = str(word).lower()
for indx in range(0,len(reposContentsList)):
     for indx2 in range(0,len(progLangResvWordsLs)):
          reposContentsList[indx]=reposContentsList[indx].replace(progLangResvWordsLs[indx2],"")    

In [0]:
#Tokenize ,stemming, lemmatize
stem=PorterStemmer()
stemmingWordsLs=[]
Lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")
lemmatizWordsLs=[]
reposContentsTokenList = []
reposContentsStrList=[]
# Remove Duplications and stemmer
for indx in range(0,len(reposContentsList)):
        reposContentsTokenList += [list(dict.fromkeys(reposContentsList[indx].split(' ')))]
        #print(list(dict.fromkeys(reposContentsList[indx].split(' '))))
        reposContentsStrList.append('')
        wordsLs=reposContentsTokenList[-1]
        #print(wordsLs)
        for wordIndx in range(0,len(wordsLs)):
            if wordsLs[wordIndx] not in gensim.parsing.preprocessing.STOPWORDS and len(wordsLs[wordIndx]) > 3: 
                    lemmatizWord=Lemmatizer.lemmatize(wordsLs[wordIndx])
                    stemmingWordsLs+=[stemmer.stem(lemmatizWord)]
                    reposContentsStrList[-1]+=stemmingWordsLs[-1]
                    if(wordIndx !=len(wordsLs)-1):
                        reposContentsStrList[-1]+=' '
          
        print(wordIndx)
        reposContentsTokenList[-1]= stemmingWordsLs
        stemmingWordsLs=[]
        lemmatizWordsLs=[]
file_1=open('/gdrive/My Drive/Colab Notebooks/stemmed_lemmarized_data.txt','w')
file_1.write(str(reposContentsStrList))

894
1663
6504
14784
1400
269
239
1770
1821
821
2264
277
140
1203
600
1177
1149
784
14101
5125
10917
1168
228
227
590
1915
2455
464
3440
7173
5213
4917
552
881
749
717
988
10295
9069
2139
6864
1122
563
264
19511
30
219
1161
132
3195
695
1874
908
515
34
197
69
510
2098
400
172
1304
258
1272
992
597
4657
178
10195
54687
9052
69
4381
373
2320
1793
176
1074
245
798
358
2579
1869
22690
2534
267
175
210
907
4730
72
1100
857
2014
1032
2432
3745
11335
177
1397
716
305
2547
1084
833
2515
1497
1751
6866
12672
10875
9723
1363
14065
26407
9576
1421
1842
25889
13890
14561
27977
920
212
6615
2048
322
3164
1110
2170
952
1159
10715
5477
4322
4842
769
212
958
178
8331
1429
85026
2219
352
49
24744
2851
213
3236
1854
488
8393
265
591
23891
1249
672
4605
9737
243
477
1715
304
209
2431
47598
28830
158690
694
14526
483
3223
669
37233
411
5679
5649
15127
1779
46537
5344
518
2445
1071
12168
2787
1289
6
2143
7254
17230
4142
6334
859
2379
239
2716
2352
1197
2307
8032
16812
36531
1704
42423
239
1197
808
836
250
1

244405620

#Data Encoding


In [0]:
# Encodeing the text by tfidf.
vectorizer = TfidfVectorizer(analyzer='word',stop_words='english')
X_encoded = vectorizer.fit_transform(cleanedData)

In [11]:
# Encoding the text by giving each token to integer number the most frequent words will be taken into account.
# Prepare tokenizer to tokenize the list of strings to list of list of tokens.
t=Tokenizer()
t.fit_on_texts(cleanedData)
# Encode the tokens to integers
vocab_size = len(t.word_index)+1
print(vocab_size)
X_encoded = t.texts_to_sequences(cleanedData)

5517900


In [0]:
X_encoded=pad_sequences(X_encoded,maxlen=len(max(X_encoded)),padding='post')

In [0]:
#This is good for machine Learning models.
labelEnc =LabelEncoder()
Y_encoded=labelEnc.fit_transform(Y)

In [0]:
# This is good for Deep learning models.Like shawllow NN.
one=OneHotEncoder()
Y_encoded=one.fit_transform(np.array(Y).reshape(-1,1))

In [0]:
# shuffle randomly.
X_encoded,Y_encoded,reposNameList=shuffle(X_encoded,Y_encoded,reposNameList)


In [0]:
# splitting the data into two parts training and testing
xTrain,xTest,yTrain,yTest,repoNaTrain,repoNaTest=train_test_split(X_encoded,Y_encoded,reposNameList,test_size=0.2)

In [0]:
# This function help us to use the train data or testing or validation efficiently by memory. I used in DeepLearning Like NN.
def data_generator(X,Y,batch_size=32):
    startB=0
    for indx in range(0,int(len(Y)/batch_size)):
        yield(X[startB:batch_size+startB],Y[startB:batch_size+startB])
        startB+=batch_size    

In [0]:
stdXTrain=(xTrain-xTrain.mean())/xTrain.std()
stdYTrain= (yTrain-yTrain.mean())/yTrain.std()
stdXTest=(xTest-xTest.mean())/xTest.std()
stdYTest= (yTest-yTest.mean())/yTest.std()

#Unsupervised Learning Methods(Clustering by Kmeans or LDA)

##CNN autoencoders and Standard autoencoder respectively

In [0]:

'''
one=OneHotEncoder()
xTrain,xTest=train_test_split(X,test_size=0.33)
xtrain=one.fit_transform(xTrain.toarray())
xtest= one.fit_transform(xTest.toarray())
#label=LabelEncoder()
#label.fit_transform(xTrain)

filtersize = (3,3)
numUnits = 10 
input_shape = xtrain.shape
print(input_shape)
model = Sequential()
model.add(Embedding(input_dim=input_shape[0],input_length=input_shape[1],output_dim=10))
#model.add(Conv1D(numUnits,3,padding='same'))
#model.add(MaxPool1D(pool_size=2))
#model.add(Conv1D(numUnits,3))
#model.add(MaxPool1D(pool_size=2))
#model.add(Conv1D(numUnits,3,padding='same'))
model.add(MaxPool1D(pool_size=3))
model.add(Conv1D(numUnits,3,activation='relu',padding='same'))
encodedModel = model
model.add(Lambda(lambda x: K.expand_dims(x, axis=2)))
model.add(Conv2DTranspose(filters=10, kernel_size=(3, 1),activation='relu',padding='same'))
model.add(Lambda(lambda x: K.squeeze(x, axis=2)))
model.add(UpSampling1D(size=3))
model.add(ZeroPadding1D(padding=1))
#model.add(Lambda(lambda x: K.expand_dims(x, axis=2)))
#model.add(Conv2DTranspose(filters=10, kernel_size=(3, 1),padding='same'))
#model.add(Lambda(lambda x: K.squeeze(x, axis=2)))
#model.add(UpSampling1D(size=2))
#model.add(Lambda(lambda x: K.expand_dims(x, axis=2)))
#model.add(Conv2DTranspose(filters=10, kernel_size=(3, 1)))
#model.add(Lambda(lambda x: K.squeeze(x, axis=2)))
#model.add(UpSampling1D(size=2))
#model.add(Lambda(lambda x: K.expand_dims(x, axis=2)))
#model.add(Conv2DTranspose(filters=10, kernel_size=(3, 1),padding='same'))
#model.add(Lambda(lambda x: K.squeeze(x, axis=2)))
model.add(Dense(1))
model.add(Lambda(lambda x: K.squeeze(x, axis=2)))
#model.add(Lambda(lambda x: K.squeeze(x, axis=1)))
#print(list(model.layers[-1].output_shape))
#newdim = tuple([x for x in list(model.layers[-1].output_shape) if x !=10 and x is not None])
#print(newdim)
#model.add(Reshape((0,newdim[0])))
model.compile(loss='mse',optimizer='Adam')
model.summary()
model.fit(xtrain,xtrain,epochs=20)
'''

In [0]:
'''
autoencoder = Sequential()
autoencoder.add(Dense(xtrain.shape[1],input_shape=xtrain.shape))
autoencoder.add(Dense(50))
autoencoder.add(Dense(50))
autoencoder.add(Dense(xtrain.shape[1]))
autoencoder.compile(loss='mse',optimizer='sgd')
autoencoder.summary()
autoencoder.fit(xtrain,xtrain,epochs=10)
# to drop all decoded layers to get the encoded layers.
# this step will use it to get encoded data from CNN it's called feature extraction.
for layer in reversed(encodedModel.layers):
     if(re.search('Conv1D',str(layer))):
        break
     encodedModel.pop()
     # add flatten layer to put them in 2 dimesion shape of the previous layer to one dimension.
     #encodedModel.layers
     encodedModel.add(Flatten())
     predictionTs=model.predict(xtest)
     #predictionTr=model.predict(xtrain)
     #encodedData.shape
     #encodedData=encodedModel.predict(xtest)
'''

## LDA (Latent Dirichlet allocation) Topic modeling

In [0]:
# Create the bag of words for each repo the word index and how many each token was repeating in each repo string.
# used for unspervised learning methods like LDA
index = 0
wordIndx = 0
bowCorpus=[[]]
print(len(reposContentsTokenList[index]))
while(wordIndx < len(reposContentsTokenList[index])):
       bowCorpus[-1]+=[(t.word_index[reposContentsTokenList[index][wordIndx]],t.word_counts[reposContentsTokenList[index][wordIndx]])]
       if(wordIndx == len(reposContentsTokenList[index])-1):
               #print(1)
               index+=1
               wordIndx = 0
               bowCorpus.append([])
       if(index == len(reposContentsTokenList)-1): 
               break       
       wordIndx+=1 
lda_model = gensim.models.LdaMulticore(bowCorpus,14,t.index_word,iterations=200,workers=3)

In [0]:
# to see what are the words related to each topic.
lda_model.print_topics()

## KMeans *Clustering*

In [0]:
true_k =[14]
sse=[]
KMeansModelsLs =[]
for indx in range(0,len(true_k)):
    KMeansModelsLs += [KMeans(n_clusters=true_k[indx],init="k-means++", max_iter=100, n_init=1,random_state=3452)]
    KMeansModelsLs[-1].fit(xtrain)
    sse.append(KMeansModelsLs[-1].inertia_)
    print(sse)
# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(true_k, sse, '-o')
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');

###Show the results

In [0]:
# Prediction for training.
termsLs =[]
for KmsModlindx in range(0,len(KMeansModelsLs)):
    print("Kmeans number of cluster",true_k[indx])
    order_centroids = KMeansModelsLs[-1].cluster_centers_.argsort()[:, ::-1]
    #print(order_centroids)
    terms = vectorizer.get_feature_names()
    #print(terms)
    for i in range(true_k[indx]):
        print("Cluster %d:" % i),
        for ind in order_centroids[i,:]:
            #print(ind)
            print(' %s' % terms[ind])
            termsLs +=[terms[ind]]
    print("\n")
    print("Prediction")
 

In [0]:
for KmsModlindx in range(0,len(KMeansModelsLs)):
       prediction = KMeansModelsLs[KmsModlindx].predict(xtest)
print(prediction)

In [0]:
# Prediction for Testing.
for KmsModlindx in range(0,len(KMeansModelsLs)):
    print("Kmeans number of cluster",true_k[indx])
    order_centroids = KMeansModelsLs[-1].cluster_centers_.argsort()[:, ::-1]
    #print(order_centroids)
    terms = vectorizer.get_feature_names()
    #print(terms)
    for clusterIndx in range(true_k[indx]):
        print("Cluster %d:" % clusterIndx),
        for ind in order_centroids[clusterIndx,50:60]:
          #print(ind)
          print(' %s' % terms[ind])
          
    print("\n")
    print("Prediction")
    prediction =  KMeansModelsLs[-1].predict(xTest[0:20])
    print(prediction) 

In [0]:
# This function will help to draw the most frequent word by put the word in large size and the less frequent with small size.
wordcloud = WordCloud(width = 1000, height = 1000, 
                background_color ='white', 
                min_font_size = 10).generate(' '.join(terms[12]))
terms=np.array(termsLs)                
terms=terms.reshape(true_k[0],xtrain.shape[1])                
plt.imshow(wordcloud)                 


# Supervised Learning Methods(Classification For Machine Learning and Deep Learning models)





## Naive Bayes

In [17]:
MNB=MultinomialNB(alpha=2.0, fit_prior=True, class_prior=None)
MNB.fit(xTrain,yTrain)
scoreTrainMNB=MNB.score(xTrain, yTrain)
yPredMNB=MNB.predict(xTest)
scoreTestMNB=accuracy_score(yTest,yPredMNB,normalize=True)*100
print("test acc",scoreTestMNB)
print("train acc",scoreTrainMNB)
#Results
#testing acc 14.874551971326163
#training acc 0.568904593639576 (56%)

test acc 10.15625
train acc 0.2446183953033268


##Logistic Regression


In [18]:
lgrCV=LogisticRegressionCV(n_jobs=10,max_iter=500)
lgrCV.fit(xTrain,yTrain) #77% training 15 to 19% test
scoreTrainlgrCV=lgrCV.score(xTrain,yTrain)
yPredlgrCV=lgrCV.predict(xTest)
scoreTestlgrCV=accuracy_score(yTest,yPredlgrCV,normalize=True)*100
print("test acc",scoreTestlgrCV)
print("train acc",scoreTrainlgrCV)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to conve

test acc 8.854166666666668
train acc 0.8414872798434442


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


##Support Vector Machines

In [19]:
'''
using Validation splitting and Randomized Search.
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(svm.SVC(), param_grid=param_grid, cv=cv)
grid.fit(xTrain, yTrain)
'''
SVM=svm.SVC(C=3.0,gamma='scale',random_state=42,kernel='poly',class_weight='balanced')
SVM.fit(xTrain,yTrain) #77% training 15 to 19% test
scoreTrainSVM=SVM.score(xTrain,yTrain)
yPredSVM=SVM.predict(xTest)
scoreTestSVM=accuracy_score(yTest,yPredSVM,normalize=True)*100
print("test acc",scoreTestSVM)
print("train acc",scoreTrainSVM)

test acc 11.458333333333332
train acc 0.33007175472928896


##Random Forest Classifier

In [20]:
'''
By using Validaiton spliting and Randomized Search
n_estimators_v=[800,2000]
max_depth_v =  [800,2000]
param_grid = dict(n_estimators=n_estimators_v, max_depth=max_depth_v )
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.33, random_state=42)
RFCGrid = GridSearchCV(RandomForestClassifier(bootstrap=True,max_features=1500000), param_grid=param_grid, cv=cv)
RFCGrid.fit(xTrain, yTrain)
'''
RFC=RandomForestClassifier(n_estimators=1200,max_depth=1000,max_features=1500,bootstrap=True,random_state=42)
RFC.fit(xTrain,yTrain)
scoreTrainRFC=RFC.score(xTrain,yTrain)
yPredRFC = RFC.predict(xTest)
scoreTestRFC=accuracy_score(yTest,yPredRFC,normalize=True)*100
print("test acc",scoreTestRFC)
print("train acc",scoreTrainRFC)

test acc 53.90625
train acc 0.8480104370515329


##Multilayer Perceptron

In [21]:
mlp = MLPClassifier(hidden_layer_sizes=(100,),learning_rate='adaptive',validation_fraction=.2,solver='lbfgs',batch_size=100,random_state=42,verbose=1)
#22 % training 20% test using hidden_layers 10 and batch_size = 100
mlp.fit(xTrain,yTrain)
yPredMlp=mlp.predict(xTest)
scoreTrainMlp=mlp.score(xTrain,yTrain)
scoreTestMlp=accuracy_score(yPredMlp,yTest)

print("test acc ",scoreTestMlp)
print("train acc ",scoreTrainMlp)

test acc  0.06510416666666667
train acc  0.27984344422700586


##Bagging Classifier


In [22]:
BC=BaggingClassifier(base_estimator=None, n_estimators=1200, max_samples=1.0, max_features=200, bootstrap=True, bootstrap_features=True, oob_score=False, warm_start=False, n_jobs=100, random_state=42, verbose=0)
BC.fit(xTrain,yTrain)
scoreTrainBC=BC.score(xTrain,yTrain)
yPredBC=BC.predict(xTest)
scoreTestBC=accuracy_score(yPredBC,yTest)
print("test acc",scoreTestBC)
print("train acc",scoreTrainBC)

test acc 0.6119791666666666
train acc 0.8460534898891063


In [23]:
numOfRepos = 1
for indx in range(0,numOfRepos):
   print('Repo Name:')
   print(repoNaTest[indx])
   print('num of class:')
   print(yPredBC[indx])
   for indx2 in range(0,len(labels)):
       if(yPredBC[indx]==indx2):
          print('Topic:')
          print(labels[indx2])
        

Repo Name:
insurance-orders
num of class:
7
Topic:
Healthcare


In [0]:
     
json.dump(BC.get_params(),open('BaggingClassifier_900_100_tr86_ts61','w'))  

##XGBoost Classifier

In [24]:
alg = XGBClassifier(learning_rate=0.0001, n_estimators=700, max_depth=700,
                        min_child_weight=3, gamma=0.2, subsample=1.0, colsample_bytree=1.0,
                        objective='multi:softmax', nthread=4, scale_pos_weight=1, seed=27,num_class=14)
xgbParams = alg.get_xgb_params()
xgbTrain = xgb.DMatrix(xTrain,label=yTrain)
cvResults=xgb.cv(xgbParams,xgbTrain,nfold=5,stratified=True,num_boost_round=alg.get_params()['n_estimators'])
alg.set_params(n_estimators=cvResults.shape[0])
alg.fit(xTrain,yTrain,eval_metric='auc')
scoreTrainXGB=alg.score(xTrain,yTrain)
yPredXGB=alg.predict(xTest)
scoreTestXGB=accuracy_score(yPredXGB,yTest,normalize=True)*100
print("test acc",scoreTestXGB)
print("train acc",scoreTrainXGB)

test acc 43.75
train acc 0.7273320287018917


##Shallow Neural Network

###The part of encoding before feeding into that model by using Glove dictionary

In [0]:
f = open('/gdrive/My Drive/Colab Notebooks/glove/glove.6B.100d.txt','r')
embedding_index = {}
# Create the dictionary of glove data key is the word : and value is the coefs
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:],dtype='float32')
  embedding_index[word] = coefs

In [0]:
# Create the embedding matrix which contains the weights  of each word of our data and the related words has the same meaning.
embedding_matrix=np.zeros((vocab_size,100))
for word,indx in t.word_index.items():
    embedding_vector =embedding_index.get(word)
    if(embedding_vector is not None):
         embedding_matrix[indx] = embedding_vector

In [0]:
Xtrain,xVal,yTrain,yVal=train_test_split(xTrain,yTrain,test_size=0.2)

In [0]:
#Simple Neural nets by using Keras API.
NN = Sequential()
NN.add(Embedding(vocab_size,100,weights=[embedding_matrix],input_length=xTrain.shape[1]))
NN.add(GRU(50,activation='relu'))
NN.add(Dense(14,activation='softmax')) 
NN.compile(optimizer='adam',loss='categorical_crossentropy')
NN.summary()
NN.fit_generator(data_generator(xTrain,yTrain,1),validation_data=data_generator(xVal,yVal,1),steps_per_epoch=1,epochs=100,validation_steps=1)
#NN.fit(xTrain,yTrain,validation_split=0.33,epochs=50,batch_size=100)
#NN.predict_generator()
yPredNN=NN.predict(xTest)
scoreNNTrain=NN.evaluate(xTrain,yTrain)
#scoreNNTest=NN.evaluate(xTest,yTest)
scoreTestNN=accuracy_score(yPredNN,yTest,normalize=True)*100
print("test acc",scoreTestNN)
print("train acc",scoreTrainNN)

In [0]:
for x,y in data_generator(xTrain,yTrain,1):
   print(x,y)
   break